### Our Script so far

In [27]:
from csv import reader
import dataset
import pickle

data_rdr = reader(open('mn.csv', 'r', encoding='utf-8'))
header_rdr = reader(open('mn_headers_updated.csv', 'r', encoding='utf-8'))

data_rows = [d for d in data_rdr]
header_rows = [h for h in header_rdr if h[0] in data_rows[0]]

all_short_headers = [h[0] for h in header_rows]
skip_index = []
final_header_rows = []

for header in data_rows[0]:
    if header not in all_short_headers:
        print(header)
        index = data_rows[0].index(header)
        if index not in skip_index:
            skip_index.append(index)
    else:
        for head in header_rows:
            if head[0] == header:
                final_header_rows.append(head)
                break
new_data = []

for row in data_rows[1:]:
    new_row = []
    
    for i, d in enumerate(row):
        if i not in skip_index:
            new_row.append(d)
    new_data.append(new_row)

zipped_data = []
for drow in new_data:
    zipped_data.append(zip(final_header_rows, drow))
    
# Save the list to a file
with open('zipped_data.pkl', 'wb') as file:
    pickle.dump(zipped_data, file)


mwelevel
mnweight
wscoreu
windex5u
wscorer
windex5r


In [28]:
# Load the list from the file
def load_zipped_data():
    with open('zipped_data.pkl', 'rb') as file:
        zipped_data = pickle.load(file)
        return zipped_data

In [29]:
# look for missing
zipped_data = load_zipped_data()
for x in zipped_data[0]:
    if not x[1]:
        print(x)

In [30]:
#look for duplicates

set_of_keys = set()
zipped_data = load_zipped_data()
for x in zipped_data:
    unique_key = ''
    for i, row in enumerate(x):
        if i in [0, 1, 2]:
            unique_key += str(row[1])+'-'
    set_of_keys.add(unique_key[:-1])

uniques = []
zipped_data = load_zipped_data()
for x in zipped_data:
    unique_key = ''
    for i, row in enumerate(x):
        if i in [0, 1, 2]:
            unique_key += str(row[1])+'-'
    if not set_of_keys.remove(unique_key[:-1]):
        uniques.append(x)
        
print(set_of_keys)

set()


In [31]:
#save to db
zipped_data = load_zipped_data()
db = dataset.connect('sqlite:///data_wrangling.db')
table = db['unicef_survey']

for row_num, data in enumerate(zipped_data):
    for question, answer in data:
        data_dict = {
            'question': question[1],
            'question_code': question[0],
            'answer': answer,
            'response_number': row_num,
            'survery': 'mn',
        }
    table.insert(data_dict)

#### we can see most of our code is flat, meaning we don't have nested levels of importance. Much of the code and functions sit without identation or documentation in the file. It's not well abstracted, and the variable names are unclear. let's start working on parts of that, beginning at the top. The first two sets of lines repeat each other. Let's write a function to do that instead.